## save and restore `tf.Variable`'s

In [1]:
!ls /tmp

KSDownloadAction.NaMzed9wYS    mnist5
checkpoint                     model.ckpt.data-00000-of-00001
com.apple.launchd.JMMiAKTZnT   model.ckpt.index
com.apple.launchd.MfreScsdco   model.ckpt.meta
com.apple.launchd.oE3iVVkw3N   powerlog
data                           t10k-images-idx3-ubyte.gz
mnist                          t10k-labels-idx1-ubyte.gz
mnist2                         train-images-idx3-ubyte.gz
mnist3                         train-labels-idx1-ubyte.gz
mnist4


In [2]:
import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print("Model saved in path: %s" % save_path)

Instructions for updating:
Colocations handled automatically by placer.
Model saved in path: /tmp/model.ckpt


In [3]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


In [4]:
!ls -la /tmp/

total 22712
drwxrwxrwt  21 root   wheel      672 May 15 05:45 .
drwxr-xr-x   6 root   wheel      192 Apr 26  2018 ..
drwx------   2 root   wheel       64 May 15 05:43 KSDownloadAction.NaMzed9wYS
-rw-r--r--   1 aaron  wheel       87 May 15 05:45 checkpoint
drwx------   3 aaron  wheel       96 Apr  2 05:46 com.apple.launchd.JMMiAKTZnT
drwx------   3 aaron  wheel       96 Apr  2 05:46 com.apple.launchd.MfreScsdco
drwx------   3 aaron  wheel       96 Apr  2 05:46 com.apple.launchd.oE3iVVkw3N
drwxr-xr-x   3 aaron  wheel       96 Apr 23 06:11 data
drwxr-xr-x   3 aaron  wheel       96 May 11 11:36 mnist
drwxr-xr-x   4 aaron  wheel      128 May 12 07:53 mnist2
drwxr-xr-x   2 aaron  wheel       64 May 12 07:55 mnist3
drwxr-xr-x   2 aaron  wheel       64 May 12 07:56 mnist4
drwxr-xr-x   2 aaron  wheel       64 May 12 07:57 mnist5
-rw-r--r--   1 aaron  wheel       32 May 15 05:45 model.ckpt.data-00000-of-00001
-rw-r--r--   1 aaron  wheel      143 May 15 05:45 model.ckpt.index
-rw-r--r--   1 aaron

In [5]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
  # Initialize v1 since the saver will not.
  v1.initializer.run()
  saver.restore(sess, "/tmp/model.ckpt")

  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


In [6]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='', all_tensors=True)

# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v1', all_tensors=False)

# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v2', all_tensors=False)

# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]
tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]


## save and restore `tf.Model` - [docs](https://www.tensorflow.org/guide/saved_model#save_and_restore_models)